## This pattern allows the model to make decisions about which path to take through a workflow based on context and intermediate results. The model acts as an intelligent router, directing the flow of execution between different branches of your workflow. This is particularly useful when handling varied inputs that require different processing approaches. In the example below, the results of the first LLM call change the properties of the second LLM call like model size and system prompt.

In [1]:
import { generateText, generateObject } from 'ai'
import { z } from 'zod'
import { createCompatibleOpenAI } from '../../init.ts'

const openai = createCompatibleOpenAI()

const classificationSchema = z.object({
	type: z.enum(['general', 'refund', 'technical']),
	complexity: z.enum(['simple', 'medium', 'complex']),
	reasoning: z.string(),
})


async function handleCustomerQuery(query: string) {
	const model = openai('gpt-4o-mini')

	// 1.Classify the query type
	const { object: classification } = await generateObject<z.infer<typeof classificationSchema>>({
		model,
		schema: z.object({
			type: z.enum(['general', 'refund', 'technical']),
			complexity: z.enum(['simple', 'medium', 'complex']),
			reasoning: z.string(),
		}),
		prompt: `Classify the following customer query:

		${query}

		Determine:
		1. Query type(general, refund, or technical)
		2. Complexity(simple, medium, or complex)
		3. Brief reasoning for classification
		`
	})

	  // Route based on classification
  // Set model and system prompt based on query type and complexity
  const { text: response } = await generateText({
    model:
      classification.complexity === 'simple'
        ? openai('gpt-4o-mini')
        : openai('o3-mini'),
    system: {
      general:
        'You are an expert customer service agent handling general inquiries.',
      refund:
        'You are a customer service agent specializing in refund requests. Follow company policy and collect necessary information.',
      technical:
        'You are a technical support specialist with deep product knowledge. Focus on clear step-by-step troubleshooting.',
    }[classification.type],
    prompt: query,
  });

	return response
}

const query = 'I need help with my account'
const result = await handleCustomerQuery(query)
console.log(result)

I'd be happy to help you with your account! Could you please provide more details about the issue you're experiencing? Whether it's logging in, updating information, or something else, let me know how I can assist you.
